<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_2C_DataAggregation_DuckDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

References


*   https://www.enterprisedb.com/postgres-tutorials/how-use-grouping-sets-cube-and-rollup-postgresql

Free Data Sets


1.   https://www.iguazio.com/blog/13-best-free-retail-datasets-for-machine-learning/






In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2025-07-08 13:27:07.353076+05:30
Python 3.11.13
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


#Get Data

In [2]:
import pandas as pd

In [3]:
#Get Data File
!wget -q -O SS_Orders.csv 'https://raw.githubusercontent.com/Praxis-QR/RDWH/main/data/SS_Orders.csv'

In [4]:
SS_df = pd.read_csv("/content/SS_Orders.csv")
SS_df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,11/10/2015,18/10/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [5]:
SS_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [6]:
# Convert to datetime format
SS_df['Order Date'] = pd.to_datetime(SS_df['Order Date'], format='%d/%m/%Y')
SS_df['Ship Date'] = pd.to_datetime(SS_df['Ship Date'], format='%d/%m/%Y')
SS_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   datetime64[ns]
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

#Setup

In [7]:
!pip install duckdb

In [8]:
import duckdb
con = duckdb.connect()
#con.register("SS_t", SS_df)

In [9]:
con.register("ss_order", SS_df)

In [10]:
con.execute('select * from ss_order limit 3').fetchdf()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


#Group By - Basic

In [11]:
sql = "\
SELECT segment, region, category, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region, category; \
"
con.execute(sql).fetchdf()

,Segment,Region,Category,sales
0,Consumer,West,Furniture,119808.0870
1,Consumer,West,Office Supplies,110080.9400
2,Home Office,Central,Office Supplies,32777.2350
3,Corporate,Central,Office Supplies,41137.7010
4,Corporate,Central,Furniture,52085.6018
5,Home Office,Central,Furniture,25482.3430
6,Corporate,South,Office Supplies,45930.1700
7,Consumer,East,Technology,135441.2290
8,Corporate,East,Office Supplies,66474.7350
9,Corporate,West,Technology,65641.3120


In [12]:
sql = "\
SELECT segment, region, SUM (sales) sales \
FROM ss_order \
GROUP BY segment, region; \
"
con.execute(sql).fetchdf()

,Segment,Region,sales
0,Consumer,Central,252031.4340
1,Corporate,Central,157995.8128
2,Consumer,East,350908.1670
3,Corporate,East,200409.3470
4,Home Office,West,136721.7770
5,Home Office,South,74255.0015
6,Consumer,South,195580.9710
7,Corporate,West,225855.2745
8,Consumer,West,362880.7730
9,Home Office,Central,91212.6440


In [13]:
sql = "\
SELECT segment, SUM (sales) sales \
FROM ss_order \
GROUP BY segment; \
"
con.execute(sql).fetchdf()

,Segment,sales
0,Consumer,1.161401e+06
1,Corporate,7.061464e+05
2,Home Office,4.296531e+05


In [14]:
sql = "\
SELECT  SUM (sales) sales \
FROM ss_order  \
"
con.execute(sql).fetchdf()

,sales
0,2.297201e+06


#Roll Up

## Basic Roll up

In [15]:
sql = "\
SELECT category, SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
con.execute(sql).fetchdf()

,Category,sum(quantity)
0,None,37873.0
1,Furniture,8028.0
2,Office Supplies,22906.0
3,Technology,6939.0


In [16]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 SUM(quantity) \
FROM ss_order \
GROUP BY ROLLUP (category); \
"
con.execute(sql).fetchdf()

,category,sum(quantity)
0,All category,37873.0
1,Furniture,8028.0
2,Office Supplies,22906.0
3,Technology,6939.0


## MultiColumn Roll Up

In [17]:
 sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (category , segment); \
"
con.execute(sql).fetchdf()

,Category,Segment,quantity
0,Furniture,None,8028.0
1,Office Supplies,None,22906.0
2,Technology,None,6939.0
3,Office Supplies,Corporate,7018.0
4,Office Supplies,Home Office,4130.0
5,Technology,Corporate,2095.0
6,Furniture,Corporate,2495.0
7,Furniture,Home Office,1367.0
8,Technology,Home Office,1247.0
9,Furniture,Consumer,4166.0


In [18]:
# note change in order of ROLLUP
sql = "\
SELECT category, segment, SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
con.execute(sql).fetchdf()

,Category,Segment,quantity
0,None,None,37873.0
1,None,Consumer,19521.0
2,None,Corporate,11608.0
3,None,Home Office,6744.0
4,Furniture,Consumer,4166.0
5,Office Supplies,Corporate,7018.0
6,Office Supplies,Consumer,11758.0
7,Furniture,Corporate,2495.0
8,Technology,Home Office,1247.0
9,Technology,Consumer,3597.0


In [19]:
sql ="\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY ROLLUP (segment , category); \
"
con.execute(sql).fetchdf()

,category,segment,quantity
0,All category,All segment,37873.0
1,All category,Consumer,19521.0
2,All category,Corporate,11608.0
3,All category,Home Office,6744.0
4,Furniture,Consumer,4166.0
5,Office Supplies,Corporate,7018.0
6,Office Supplies,Consumer,11758.0
7,Furniture,Corporate,2495.0
8,Technology,Home Office,1247.0
9,Technology,Consumer,3597.0


## Partial Roll Up

In [20]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY segment, ROLLUP (category); \
"
con.execute(sql).fetchdf()

,category,segment,quantity
0,All category,Consumer,19521.0
1,All category,Corporate,11608.0
2,All category,Home Office,6744.0
3,Technology,Consumer,3597.0
4,Office Supplies,Home Office,4130.0
5,Technology,Corporate,2095.0
6,Furniture,Home Office,1367.0
7,Furniture,Consumer,4166.0
8,Office Supplies,Corporate,7018.0
9,Office Supplies,Consumer,11758.0


In [21]:
sql = "\
SELECT COALESCE(category, 'All category') AS category, \
 COALESCE(segment, 'All segment') AS segment, \
 SUM(quantity) quantity \
FROM ss_order \
GROUP BY category, ROLLUP (segment); \
"
con.execute(sql).fetchdf()

,category,segment,quantity
0,Technology,All segment,6939.0
1,Furniture,All segment,8028.0
2,Office Supplies,All segment,22906.0
3,Office Supplies,Corporate,7018.0
4,Office Supplies,Home Office,4130.0
5,Technology,Corporate,2095.0
6,Furniture,Corporate,2495.0
7,Furniture,Home Office,1367.0
8,Technology,Home Office,1247.0
9,Furniture,Consumer,4166.0


#Cube

In [22]:
#One Dimensional Cube
#
sql = "\
SELECT COALESCE(category, 'All category') category, SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category) \
ORDER BY category;  \
"
con.execute(sql).fetchdf()

,category,quantity
0,All category,37873.0
1,Furniture,8028.0
2,Office Supplies,22906.0
3,Technology,6939.0


In [23]:
#Two Dimensional Cube
#
sql = "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment) \
ORDER BY category,segment;  \
"
con.execute(sql).fetchdf()

,category,segment,quantity
0,All category,All segment,37873.0
1,All category,Consumer,19521.0
2,All category,Corporate,11608.0
3,All category,Home Office,6744.0
4,Furniture,All segment,8028.0
5,Furniture,Consumer,4166.0
6,Furniture,Corporate,2495.0
7,Furniture,Home Office,1367.0
8,Office Supplies,All segment,22906.0
9,Office Supplies,Consumer,11758.0


In [24]:
# Three Dimensional Cube
#
sql = "\
SELECT \
COALESCE(category, 'All category') category, \
COALESCE(segment, 'All segment') segment, \
COALESCE(region, 'All region') region, \
SUM(quantity) quantity \
FROM ss_order \
GROUP BY CUBE(category, segment, region) \
ORDER BY category,segment, region;  \
"
con.execute(sql).fetchdf()

,category,segment,region,quantity
0,All category,All segment,All region,37873.0
1,All category,All segment,Central,8780.0
2,All category,All segment,East,10618.0
3,All category,All segment,South,6209.0
4,All category,All segment,West,12266.0
...,...,...,...,...
75,Technology,Home Office,All region,1247.0
76,Technology,Home Office,Central,297.0
77,Technology,Home Office,East,354.0
78,Technology,Home Office,South,214.0


# Grouping Sets

Grouping sets allow us to define multiple grouping levels within a single query. Instead of writing multiple queries to produce the desired report, we can use grouping sets to generate the report in a single query. This can be especially useful when dealing with large data sets. because the time taken to read the data is often the bottleneck for performance. In such cases, it is desirable to read the data once and generate multiple results simultaneously. This is precisely what can be accomplished with GROUP BY GROUP SETS.

see https://stories.fylehq.com/p/demystifying-grouping-sets-in-postgres

In [25]:
#Rather confusing
sql = "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment) \
    ); \
"
con.execute(sql).fetchdf()

,Category,Region,Segment,qty
0,Office Supplies,South,Consumer,1899.0
1,Office Supplies,West,Consumer,3837.0
2,Technology,West,Consumer,1235.0
3,Office Supplies,Central,Consumer,2777.0
4,Office Supplies,East,Consumer,3245.0
5,Technology,East,Consumer,988.0
6,Furniture,Central,Consumer,952.0
7,Technology,Central,Consumer,790.0
8,Furniture,East,Home Office,377.0
9,Furniture,East,Corporate,723.0


In [26]:
sql = "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region) \
    ); \
"
con.execute(sql).fetchdf()

,Category,Region,qty
0,Office Supplies,Central,5409.0
1,Furniture,East,2214.0
2,Office Supplies,East,6462.0
3,Technology,Central,1544.0
4,Furniture,Central,1827.0
5,Technology,East,1942.0
6,Furniture,South,1291.0
7,Office Supplies,West,7235.0
8,Office Supplies,South,3800.0
9,Furniture,West,2696.0


In [27]:
sql =  "\
SELECT category,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category) \
    ); \
"
con.execute(sql).fetchdf()

,Category,qty
0,Technology,6939.0
1,Furniture,8028.0
2,Office Supplies,22906.0


In [28]:
#Rather confusing
sql =  "\
SELECT category, region,segment,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region,segment), \
        (category,region), \
        (category, segment), \
        (segment, region), \
        (category), \
        (region), \
        (segment), \
        () \
    ); \
"
con.execute(sql).fetchdf()

,Category,Region,Segment,qty
0,Furniture,South,Consumer,695.0
1,Office Supplies,West,Corporate,2097.0
2,Furniture,West,Consumer,1405.0
3,Office Supplies,Central,Home Office,1027.0
4,Office Supplies,Central,Corporate,1605.0
...,...,...,...,...
75,None,West,None,12266.0
76,None,None,Consumer,19521.0
77,None,None,Corporate,11608.0
78,None,None,Home Office,6744.0


In [40]:
sql = "\
SELECT category, region,  SUM (quantity) qty \
FROM ss_order \
GROUP BY \
    GROUPING SETS( \
        (category,region), \
        (category), \
        (region), \
        () \
    ); \
"
con.execute(sql).fetchdf()

,Category,Region,qty
0,Furniture,South,1291.00
1,Office Supplies,West,7235.00
2,Office Supplies,South,3800.00
3,Furniture,West,2696.00
4,Technology,West,2335.00
5,Technology,South,1118.00
6,Office Supplies,Central,5409.00
7,Furniture,East,2214.00
8,Office Supplies,East,6462.00
9,Technology,Central,1544.00


#Cross Tab / Pivot Tables

## Level 1

In [30]:
sql = " \
    select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Category \
    "
con.execute(sql).fetchdf()

,Category,Central,East,West,South
0,Furniture,1827.0,2214.0,2696.0,1291.0
1,Office Supplies,5409.0,6462.0,7235.0,3800.0
2,Technology,1544.0,1942.0,2335.0,1118.0


## Level 2

In [31]:
sql = " \
  select \
  COALESCE(Category, 'All Category') AS Category, \
  COALESCE(Region, 'All Region') AS Region, \
  Sum(quantity) Quantity \
  from ss_order \
  group by ROLLUP (Category,Region)  \
"
con.execute(sql).fetchdf()

,Category,Region,Quantity
0,All Category,All Region,37873.0
1,Furniture,All Region,8028.0
2,Office Supplies,All Region,22906.0
3,Technology,All Region,6939.0
4,Furniture,South,1291.0
5,Office Supplies,West,7235.0
6,Office Supplies,South,3800.0
7,Furniture,West,2696.0
8,Technology,West,2335.0
9,Technology,South,1118.0


## Level 3 - the FULL CROSSTAB with 3 rotations

In [32]:
sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Central, sums.East, sums.West, sums.South, \
sums.Central + sums.East + sums.West + sums.South  Totals \
FROM ( \
  select Category, \
    sum(case when (Region = 'Central') then quantity else 0 end) as Central, \
    sum(case when (Region = 'East') then quantity else 0 end) as East, \
    sum(case when (Region = 'West') then quantity else 0 end) as West, \
    sum(case when (Region = 'South') then quantity else 0 end) as South \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
con.execute(sql).fetchdf()

,Dim,Central,East,West,South,Totals
0,Furniture,1827.0,2214.0,2696.0,1291.0,8028.0
1,Office Supplies,5409.0,6462.0,7235.0,3800.0,22906.0
2,Totals,8780.0,10618.0,12266.0,6209.0,37873.0
3,Technology,1544.0,1942.0,2335.0,1118.0,6939.0


In [33]:
 sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Consumer, sums.Corporate, sums.HomeOffice, \
sums.Consumer + sums.Corporate + sums.HomeOffice   Totals \
FROM ( \
  select Category, \
    sum(case when (Segment = 'Consumer') then sales else 0 end) as Consumer, \
    sum(case when (Segment = 'Corporate') then sales else 0 end) as Corporate, \
    sum(case when (Segment = 'Home Office') then sales else 0 end) as HomeOffice \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
con.execute(sql).fetchdf()

,Dim,Consumer,Corporate,HomeOffice,Totals
0,Furniture,391049.312,229019.7858,121930.6975,7.419998e+05
1,Office Supplies,363952.136,230676.4620,124418.4340,7.190470e+05
2,Totals,1161401.345,706146.3668,429653.1485,2.297201e+06
3,Technology,406399.897,246450.1190,183304.0170,8.361540e+05


In [34]:
sql = " \
SELECT \
COALESCE(Category, 'Totals') AS Dim, \
sums.Central, sums.East, sums.West, sums.South, \
sums.Central + sums.East + sums.West + sums.South  Totals \
FROM ( \
  select Category, \
    sum(case when (Region = 'Central') then sales else 0 end) as Central, \
    sum(case when (Region = 'East') then sales else 0 end) as East, \
    sum(case when (Region = 'West') then sales else 0 end) as West, \
    sum(case when (Region = 'South') then sales else 0 end) as South \
    from ss_order \
    group by Rollup (Category) \
) AS sums \
"
con.execute(sql).fetchdf()

,Dim,Central,East,West,South,Totals
0,Totals,501239.8908,678781.240,725457.8245,391721.905,2.297201e+06
1,Furniture,163797.1638,208291.204,252612.7435,117298.684,7.419998e+05
2,Office Supplies,167026.4150,205516.055,220853.2490,125651.313,7.190470e+05
3,Technology,170416.3120,264973.981,251991.8320,148771.908,8.361540e+05


## Verify L3 Pivot with Python Pandas

In [35]:
dfSS = pd.read_csv('SS_Orders.csv', delimiter = ',')
dfSS.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,08/11/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820
2,3,CA-2016-138688,12/06/2016,16/06/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.0,6.8714


In [36]:
dfSS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

In [37]:
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [38]:
pd.pivot_table(data=dfSS,index='Category',columns='Segment',values='Sales', aggfunc='sum', margins=True)

Segment,Consumer,Corporate,Home Office,All
Category,,,,
Furniture,391049.31,229019.79,121930.70,741999.80
Office Supplies,363952.14,230676.46,124418.43,719047.03
Technology,406399.90,246450.12,183304.02,836154.03
All,1161401.34,706146.37,429653.15,2297200.86


In [39]:
from datetime import datetime
import pytz
print(datetime.now(pytz.timezone('Asia/Calcutta')))


2025-07-08 13:27:17.621873+05:30


#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)